In [52]:
# Imports
import os
import shutil
import cv2
import numpy as np
from tqdm import tqdm

### Segregate Data into Train, validation and test Sets

In [26]:
# def get_train_val_test_data(arr):
#         g_files = [x for x in arr if '-G-' in x]
#         f_files = [x for x in arr if '-F-' in x]
        
#         g_train_num = int(len(g_files)*0.7)

#         g_train = g_files[:g_train_num]
#         g_temp = g_files[g_train_num:]

#         g_val_nums = len(g_temp) // 2
#         g_val = g_temp[:g_val_nums]
#         g_test = g_temp[g_val_nums:]

#         f_train_num = int(len(f_files)*0.7)

#         f_train = f_files[:f_train_num]
#         f_temp = f_files[f_train_num:]

#         f_val_nums = len(f_temp) // 2
#         f_val = f_temp[:f_val_nums]
#         f_test = f_temp[f_val_nums:]

#         return g_train, g_val, g_test, f_train, f_val, f_test

In [38]:
# data_dir = '..\\data'
# train_dir = "train_data"
# val_dir = "val_data"
# test_dir = "test_data"
# count = 0
# for language in os.listdir(data_dir):
#     language_dir = os.path.join(data_dir, language)
#     # print(language_dir)
#     if os.path.isdir(language_dir):
#         for user in os.listdir(language_dir):
#             user_dir = os.path.join(language_dir, user)
#             # print(user_dir)
#             files = list(os.listdir(user_dir))
#             # print(files)
#             g_train, g_val, g_test, f_train, f_val, f_test = get_train_val_test_data(files)
            
#             for data_type in [g_train, g_val, g_test, f_train, f_val, f_test]:
#                 src_folder = user_dir
#                 if data_type in [g_train, f_train]:
#                     os.makedirs(os.path.join(data_dir, train_dir), exist_ok=True)
#                     tmp_folder = os.path.join(data_dir, train_dir)
#                     if data_type == g_train:
#                         os.makedirs(os.path.join(tmp_folder, 'genuine'), exist_ok=True)
#                         dest_folder = os.path.join(tmp_folder, 'genuine')
#                     else:
#                         os.makedirs(os.path.join(tmp_folder, 'forge'), exist_ok=True)
#                         dest_folder = os.path.join(tmp_folder, 'forge')
                        
#                 elif data_type in [g_val, f_val]:
#                     os.makedirs(os.path.join(data_dir, val_dir), exist_ok=True)
#                     tmp_folder = os.path.join(data_dir, val_dir)
#                     if data_type == g_val:
#                         os.makedirs(os.path.join(tmp_folder, 'genuine'), exist_ok=True)
#                         dest_folder = os.path.join(tmp_folder, 'genuine')
#                     else:
#                         os.makedirs(os.path.join(tmp_folder, 'forge'), exist_ok=True)
#                         dest_folder = os.path.join(tmp_folder, 'forge')

#                 elif data_type in [g_test, f_test]:
#                     os.makedirs(os.path.join(data_dir, test_dir), exist_ok=True)
#                     tmp_folder = os.path.join(data_dir, test_dir)
#                     if data_type == g_test:
#                         os.makedirs(os.path.join(tmp_folder, 'genuine'), exist_ok=True)
#                         dest_folder = os.path.join(tmp_folder, 'genuine')
#                     else:
#                         os.makedirs(os.path.join(tmp_folder, 'forge'), exist_ok=True)
#                         dest_folder = os.path.join(tmp_folder, 'forge')
            
            
#                 # Copy each file in the list to the destination folder

#                 for file in data_type:
#                     src_string = os.path.join(src_folder, file)
#                     # if string == dest_folder:
#                     # print(string, dest_folder)
#                     shutil.copy(src_string, dest_folder)
#                     count += 1

### Preprocessing

In [104]:
filepath = '..//data//train_data//genuine//B-S-1-G-01.tif'

In [105]:
img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (150, 150))
# img = img / 255.0  # Normalize pixel values
print(img)
img = np.asarray(img)
print()
print(img)
print()
X = [img]
print(X)
# y = [1]
# X, y = np.array(X), np.array(y)
# cv2.imshow("Image", img)
# # Wait for the user to press a key
# cv2.waitKey(0)
 
# # Close all windows
# cv2.destroyAllWindows()

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]

[array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)]


In [61]:
print(f'dtype: {img.dtype}, shape: {img.shape}, min: {np.min(img)}, max: {np.max(img)}')

dtype: uint8, shape: (150, 150), min: 0, max: 255


In [101]:
# Function to load and preprocess images
# Change to Grayscale
# Resize
# Normalize
def preprocess_images(directory, label, img_size=(150, 150)):
    X = []
    y = []
    for filename in os.listdir(directory):
        if filename.endswith(".tif"):
            img = cv2.imread(os.path.join(directory, filename), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            # print(img)
            img = img / 255.0  # Normalize pixel values
            # print(img)
            img = np.asarray(img)
            # print(img)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

In [102]:
# Load and preprocess data

def get_preprocessed_data(directory):
    X = []
    y = []
    for i, data_type in tqdm(enumerate(os.listdir(data_dir)), total=len(os.listdir(data_dir)), desc='Progress...'):
        type_dir = os.path.join(data_dir, data_type)
        if os.path.isdir(type_dir):
            # print(type_dir)
            # Load genuine signatures
            if 'genuine' in type_dir:
                genuine_X, genuine_y = preprocess_images(type_dir, label=1)
                # print(genuine_X)
                # return
                X.extend(genuine_X)
                y.extend(genuine_y)
                
            else:
                forged_X, forged_y = preprocess_images(type_dir, label=0)
                X.extend(forged_X)
                y.extend(forged_y)
        
    X = np.array(X)
    y = np.array(y)
    shuffled_indices = np.random.permutation(X.shape[0])

    # Shuffle arrays using shuffled indices
    X_shuffled = X[shuffled_indices]
    y_shuffled = y[shuffled_indices]

    return X_shuffled, y_shuffled

In [103]:
data_dir = "..\\data\\train_data"
X_train, y_train = get_preprocessed_data(directory=data_dir)

Progress...:   0%|                                                                                                                          | 0/2 [00:00<?, ?it/s]

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 

Progress...:  50%|█████████████████████████████████████████████████████████                                                         | 1/2 [00:28<00:28, 28.89s/it]

[[[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 ...

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]]


TypeError: cannot unpack non-iterable NoneType object